# Problemas

Como vimos acima, há muitos passos na criação e definição de uma nova rede neural.
A grande parte desses ajustes dependem diretamente do problemas.

Abaixo, listamos alguns problemas. Todos os problemas e datasets usados vem do [Center for Machine Learning and Intelligent Systems](http://archive.ics.uci.edu/ml/datasets.php).


**Seu objetivo é determinar e implementar um modelo para cada problema.**

Isso inclui:

1. definir uma arquitetura.
Por enquanto usando somente camadas [Lineares](https://pytorch.org/docs/stable/nn.html#linear), porém podemos variar as ativações, como [Sigmoid](https://pytorch.org/docs/stable/nn.html#sigmoid), [Tanh](https://pytorch.org/docs/stable/nn.html#tanh), [ReLU](https://pytorch.org/docs/stable/nn.html#relu), [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html), [ELU](https://pytorch.org/docs/stable/generated/torch.nn.ELU.html), [SeLU](https://pytorch.org/docs/stable/generated/torch.nn.SELU.html), [PReLU](https://pytorch.org/docs/stable/generated/torch.nn.PReLU.html), [RReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html)
2. definir uma função de custo. Algums opções que vimos previamente incluem[L1](https://pytorch.org/docs/stable/nn.html#l1loss), [L2/MSE](https://pytorch.org/docs/stable/nn.html#mseloss), [Huber/SmoothL1](https://pytorch.org/docs/stable/nn.html#smoothl1loss), [*Cross-Entropy*](https://pytorch.org/docs/stable/nn.html#crossentropyloss), [Hinge](https://pytorch.org/docs/stable/nn.html#hingeembeddingloss)), e
3. definir um algoritmo de otimização ([SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [RMSProp](https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop), [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam))

A leitura do dado assim como a função de treinamento já estão implementados para você.

# Preâmbulo

In [ ]:
# imports basicos
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import os
import sys
import time
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

In [ ]:
# funções básicas

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Torch data loader"""
    if type(features) != torch.tensor:
        features = torch.tensor(features)
    if type(labels) != torch.tensor:
        labels = torch.tensor(labels)
    dataset = torch.utils.data.TensorDataset(features, labels)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)

def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = 0, 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y.long())
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size(0)

    return acc_sum / n, l.item() / len(data_iter)
  
    
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs, type='regression'):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            if type == 'regression':
              l = loss(y_hat, y.float())
            else:
              l = loss(y_hat, y.long())
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size(0)
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        if type == 'regression':
          print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                % (epoch + 1, train_l_sum / len(train_iter), test_loss, time.time() - start))
        else:
          print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))
          
        
# funcao usada para teste
def test(net, test_iter):
    print('testing on', device)
    first = True
    for X in test_iter:
        X = X.to(device)
        y_hat = net(X)
        if first is True:
            pred_logits = y_hat
            pred_labels = y_hat.argmax(axis=1)
            first = False
        else:
            pred_logits = torch.concat(pred_logits, y_hat, dim=0)
            pred_labels = torch.concat(pred_labels, y_hat.argmax(axis=1), dim=0)

    return pred_logits.numpy(), pred_labels.numpy()

# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

# Problema 1

Neste problema, você receberá 14 *features* coletadas de pacientes e tentará predizer se eles tem algum sinal de doença cardíaca. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
data = np.genfromtxt('processed.cleveland.data', delimiter=',', dtype=np.float32)
data = np.nan_to_num(data)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.20, random_state=42)
  
batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [ ]:
# IMPLEMENTE AQUI

# Problema 2

Neste problema, você receberá 90 *features* extraídas de diversas músicas (datadas de 1922 até 2011) e deve predizer o ano de cada música. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD

In [ ]:
# download do dataset
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
!unzip YearPredictionMSD.txt.zip
data = np.genfromtxt('YearPredictionMSD.txt', delimiter=',', dtype=np.float32)

print(data[0, :])
X, y = data[:, 1:], data[:, 0]
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)
  
batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [ ]:
# IMPLEMENTE AQUI

In [ ]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = net(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])

# Problema 3

Neste problema, você receberá várias *features* (como altura média, inclinação, etc) descrevendo uma região e o modelo deve predizer qual o tipo da região (floresta, montanha, etc). Mais informações sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/covertype

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip covtype.data.gz
data = np.genfromtxt('covtype.data', delimiter=',', dtype=np.float32)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)
train_labels = train_labels - 1
test_labels = test_labels - 1
  
batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [ ]:
# IMPLEMENTE AQUI

# Problema 4

Neste problema, você receberá 11 *features* extraídas de tipos de vinhos, e terá que predizer um *score* para cada vinho. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [ ]:
# download do dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
data_red = np.genfromtxt('winequality-red.csv', delimiter=';', dtype=np.float32, skip_header=1)
data_white = np.genfromtxt('winequality-white.csv', delimiter=';', dtype=np.float32, skip_header=1)
data = np.concatenate((data_red, data_white), axis=0)
data = np.nan_to_num(data)

print(data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [ ]:
# IMPLEMENTE AQUI

In [ ]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = net(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])